# Install Dependencies

In [ ]:
!pip install -q --upgrade datasets huggingface_hub fsspec rouge_score

# Load Model From HF

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from pprint import pprint
from rouge_score import rouge_scorer

In [ ]:
# define summarizer
summarizer = pipeline('summarization',
                     model='facebook/bart-large-cnn')
dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:3]')


print("_" * 50)
print(dataset)

In [ ]:
# Summarization
#--------------

# Defien score metric
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


# Summarization
for i, article in enumerate(dataset):
    text = article['article'][:1000]
    reference = article['highlights']
    summary = summarizer(text, max_length=100, min_length=30, do_sample=False)[0]["summary_text"]
    # Check score
    scores = scorer.score(text, summary)

    print(f"\n--- Sample {i+1} ---")
    pprint(f"Original Text: {text[:100]}...")
    pprint(f"Reference Summary: {reference}")
    pprint(f"Summary: {summary}")
    print("ROUGE Scores:")
    for metric, score in scores.items():
        print(f"{metric}: P={score.precision:.2f}, R={score.recall:.2f}, F1={score.fmeasure:.2f}")